# Setup

In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import glob
import nltk
import matplotlib.pyplot as plt
import json
from tqdm import tqdm_notebook
from nltk.sentiment.vader import SentimentIntensityAnalyzer

/home/gasia/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


# Data

In [3]:
# df = pd.read_csv('all_data_pos_neg_price_change.csv', parse_dates =(['timestamp'])).set_index('timestamp')
# df = df[df['message_author'] != 'BTC_Markets']
# all_messages =df
# all_messages.sort_index(inplace=True)
# all_messages.drop_duplicates(inplace=True)
# np.sum(all_messages['price_change'].notnull().values)
# all_messages = all_messages[all_messages['price_change'].notnull().values]

In [3]:
df = pd.read_csv('all_data_pos_neg_price_change_filtered_nan.csv', parse_dates =(['timestamp'])).set_index('timestamp')

In [4]:
df.sort_index(inplace=True)

In [5]:
len(df)

351666

-----------

In [6]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')  #match any word characters until it reaches a non-word character, like a space.

from stop_words import get_stop_words
# create English stop words list
en_stop = get_stop_words('en')

from nltk.stem.porter import PorterStemmer
# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
# Create lemmatizer
wnl = WordNetLemmatizer()

from collections import Counter
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/gasia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Sentence to word id

In [7]:
def file_to_word_ids(data, word_to_id):
    return [word_to_id[word] for word in data if word in word_to_id]

------------

# Dictionary of StarSpace vocabulary

In [19]:
word_space_df = pd.read_csv('/home/gasia/StarSpace/words_space_2.tsv', sep='\t', header=None)

In [20]:
word_space_df[0] = [ww[8:] if type(ww) != float else str(ww) for ww in word_space_df[0] ]
words_dict = word_space_df[0].to_dict()
words_dict = dict(zip(words_dict.values(), words_dict.keys()))

# import json 
# with open('word_dict_2.json', 'w') as outfile:
#     json.dump(words_dict, outfile)

with open('word_dict_2.json') as json_data:
    words_dict_test = json.load(json_data)

# Sentiment Analysis

In [21]:
all_messages = df
len(all_messages)

351666

In [22]:
texts = []
texts_id =[]

remove_words = ['will','s' ,'bitcoin', 'just', 'get', 'use', 'now', 'solidx' 'people','bitfinex', 'think', 'maybe', 'imageshack', 'pt', 'em', 'img', 'nbsp', 't', 'com', 'http', 'amp', '1xnk8bc', 'href', 'oto', 'www', 'isn', 'etc', 'etf', 'tr', 'td', 'img', 'ath', 'xt', 'xp', 'php', 'img', 'gt', 'pboc', 'th', 'mtgox', 'cny', 'huobi',
 'm', 'import', 'st', 'lt', 'zhou' 'ok', 'color', 'can', "adam", "bitcoin", 'import', '\xa0', 'nan', 'http', 'https', 'li', 'b', 'style' , 'font', 're', 'le', 'gif','span','hr', 'd' , 'jpg', 'png',  'am5om', 'fud', 'mt', 'th' 'hfebupaeo', 'ftdata', 'zbb', 'imgur', 'bite', 'uztgwi', 'podomatic']

map_words = {
    'btc': 'bitcoin',
    'bcc': 'bitcoin',
    'gbtc': 'bitcoin',
    'bitcoinca': 'bitcoin',
    'better': 'good',
    'increase': 'rise',
    'miner': 'mine',
    'winner': 'win'
}
ii = 0
# bow_dict = {}
# all_messages['cleaned_message'] =''
# all_messages['word_to_vec_id'] = ''

with open('word_dict_2.json') as json_data:
    word_to_id = json.load(json_data)

for m_index in tqdm_notebook(range(len(all_messages))):
    m = all_messages['message_text'][m_index]
    m =str(m)
    raw = m.lower()
    tokens = tokenizer.tokenize(raw)
    
    stopped_tokens = [i for i in tokens if not i in en_stop]
    stopped_tokens = [i for i in stopped_tokens if not i in remove_words]

    #   lemmatize tokents:  
    lemmatized_tokens = [wnl.lemmatize(i, 'a') for i in stopped_tokens]
    lemmatized_tokens = [wnl.lemmatize(i, 'n') for i in lemmatized_tokens]
    lemmatized_tokens = [wnl.lemmatize(i, 'v') for i in lemmatized_tokens]
    lemmatized_tokens = [wnl.lemmatize(i, 'r') for i in lemmatized_tokens]
    lemmatized_tokens = [wnl.lemmatize(i, 's') for i in lemmatized_tokens]
    
    #     stemmed_tokens = [p_stemmer.stem(i) for i in lemmatized_tokens]
    
    #   remove words:
    stemmed_tokens = [i for i in lemmatized_tokens if not i in remove_words]
    stemmed_tokens = [i for i in stemmed_tokens if not i in en_stop]
    d = pd.DataFrame({'z': stemmed_tokens})
    stemmed_tokens = d.replace(map_words)['z'].tolist()
    
    #   remove numeric values:
    alpha_only = [''.join(filter(str.isalpha, i))  for i in stemmed_tokens if len(''.join(filter(str.isalpha, i))) >1]
    
    if(len(alpha_only) > 0):
        texts.append(alpha_only)
        to_id = file_to_word_ids(alpha_only, word_to_id) 
        texts_id.append(to_id)
    else:
        texts.append([])
        texts_id.append([])
        
# all_messages['cleaned_message'][m_index] = alpha_only
# to_id = file_to_word_ids(alpha_only, word_to_id) 
# all_messages['word_to_vec_id'][m_index] = to_id

HBox(children=(IntProgress(value=0, max=351666), HTML(value='')))

In [23]:
all_messages['cleaned_message'] = texts
all_messages['word_to_vec_id'] = texts_id

In [24]:
all_messages.to_csv('final_ready_all_data_word_to_vec_2.csv')

In [25]:
all_messages

,category_id,message_author,message_number,message_text,mins_after,mins_before,polarity,price_change,quoteheader,subjectivity,topic_id,topic_title,cleaned_message,word_to_vec_id
timestamp,,,,,,,,,,,,,,
2016-01-01 00:57:00,57,elux,390,2016!!!!!! We made it. Bitcoin is dead. ...,430.588333,430.816000,-0.014773,-0.000529,NaN,0.427273,1061780,Rally!!!!!,"[make, dead, new, year, quote]","[9, 785, 56, 7, 1197]"
2016-01-01 01:00:00,57,The Pharmacist,391,"A'ight bitch, you're quoted. Are you laying m...",430.628333,430.619000,0.000000,0.000022,"Quote from: elux on January 01, 2016, 12:57:06...",0.000000,1061780,Rally!!!!!,"[ight, bitch, quote, lay, money, prediction]","[21275, 3616, 1197, 2383, 13, 195]"
2016-01-01 01:21:00,57,keystroke,3,Cool! Looking forward to your comments. :),430.336250,430.723000,0.468750,-0.000898,NaN,0.825000,1310368,"Another way of looking at the halving, market ...","[cool, look, forward, comment]","[1218, 64, 648, 758]"
2016-01-01 01:31:00,57,Hugroll,260,i dont think its going to go down anytime soon...,430.347917,430.444000,0.114815,-0.000223,"Quote from: roldstin on November 27, 2015, 04:...",0.262963,1266304,When will this bitcoin will go down?,"[dont, go, go, anytime, soon, halfing, also, g...","[147, 1, 1, 592, 94, 4178, 21, 394, 11, 68, 7,..."
2016-01-01 01:52:00,57,prodigy8,71,If we check the last halving period then we ca...,430.753000,430.756250,0.228348,-0.000008,"Quote from: avw1982 on December 28, 2015, 12:4...",0.517262,1304141,What will happen if there's no price rise for ...,"[check, last, halve, period, see, price, didn,...","[422, 81, 103, 329, 8, 0, 331, 110, 43, 103, 1..."
2016-01-01 03:10:00,7,noone000,72,"Bitcoin, in my opinion, will probably not repl...",430.936500,430.860000,0.093750,0.000178,NaN,0.406250,1237124,POP QUIZ: Do you believe that BTC is the world...,"[opinion, probably, replace, mean, transaction...","[180, 140, 536, 73, 99, 1258, 140, 82, 2279, 4..."
2016-01-01 03:31:00,57,kapguy,261,Why does everyone think the halving will creat...,431.659000,431.885000,0.000000,-0.000523,NaN,0.533333,1266304,When will this bitcoin will go down?,"[everyone, halve, create, instant, jump, price...","[137, 103, 169, 1453, 461, 0, 62, 5116, 126, 0..."
2016-01-01 05:11:00,57,r3t4rD4life,1,Buy and sell of a million+ coins at a time. Tu...,436.074167,436.019500,0.100000,0.000125,NaN,1.000000,1310608,"We all know huobi volume is faked, but what is...","[buy, sell, million, coin, time, turn, profit,...","[5, 16, 271, 23, 3, 368, 39, 21, 1134, 59395]"
2016-01-01 05:13:00,7,pitham1,8,"When you say next year, do you mean 2016 or 20...",436.005833,436.067000,-0.140625,-0.000140,"Quote from: Cabdinsard on December 30, 2015, 1...",0.609375,1304240,Bitcoin Price Slides 10% In PostXmas Selloff A...,"[say, next, year, mean, people, disappoint, pr...","[28, 68, 7, 73, 2, 1202, 0, 1611]"
